In [147]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import random
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
import warnings

warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold

#表示するテーブルの最大行と最大列を指定
pd.options.display.max_columns = 100
pd.options.display.max_rows = 999

seed = 42
# 乱数初期化
random.seed(seed)
np.random.seed(seed)

# データの読み込み
train = pd.read_csv('src/train.csv', index_col=0)
test = pd.read_csv('src/test.csv', index_col=0)
sample_submit = pd.read_csv('src/sample_submit.csv')

In [148]:
# 訓練データの欠損値の個数をカウント
print('train data')
print(train.isnull().sum())

train data
Age         0
Gender      0
T_Bil       0
D_Bil       0
ALP         0
ALT_GPT     0
AST_GOT     0
TP          0
Alb         0
AG_ratio    4
disease     0
dtype: int64


In [149]:
# テストデータの欠損値の個数をカウント
print('test data')
print(test.isnull().sum())

test data
Age         0
Gender      0
T_Bil       0
D_Bil       0
ALP         0
ALT_GPT     0
AST_GOT     0
TP          0
Alb         0
AG_ratio    0
dtype: int64


In [150]:
test

,Age,Gender,T_Bil,D_Bil,ALP,ALT_GPT,AST_GOT,TP,Alb,AG_ratio
id,,,,,,,,,,
891,65,Female,0.7,0.2,162.0,24.0,20.0,6.4,3.3,0.93
892,46,Male,1.2,0.3,265.1,40.0,28.0,7.9,3.8,0.94
893,26,Male,0.7,0.1,243.1,21.1,22.9,5.3,2.2,0.60
894,38,Male,3.5,1.7,253.0,80.0,406.0,6.8,3.7,1.33
895,46,Female,0.7,0.2,236.0,10.1,13.0,6.8,3.0,0.74
896,50,Male,2.7,1.0,610.0,17.0,28.0,7.1,2.6,0.59
897,43,Male,0.8,0.1,169.8,22.0,18.0,6.4,3.3,0.93
898,33,Male,0.8,0.3,187.2,22.9,43.0,7.3,3.1,0.75
899,49,Male,1.1,0.3,208.0,29.8,25.0,5.9,2.8,0.92


In [151]:
sample_submit

,891,0
0,892,0
1,893,1
2,894,1
3,895,1
4,896,1
5,897,1
6,898,1
7,899,1
8,900,0
9,901,0


In [152]:
# データ処理のためtrainとtestを結合
train_data = pd.concat([train, test])
print('trainサイズ:', len(train))
print('testサイズ:', len(test))
print('train_dataサイズ:', len(train_data))

trainサイズ: 891
testサイズ: 382
train_dataサイズ: 1273


In [153]:
train_data.dtypes

Age           int64
Gender       object
T_Bil       float64
D_Bil       float64
ALP         float64
ALT_GPT     float64
AST_GOT     float64
TP          float64
Alb         float64
AG_ratio    float64
disease     float64
dtype: object

In [154]:
# オブジェクトカラムをラベルエンコーディング
object_columns = train_data.select_dtypes(include=['object']).columns.tolist()
le = LabelEncoder()
for column in object_columns:
    train_data[column] = le.fit_transform(train_data[column])
    # train[column] = train[column].astype('cateogyr')
train_data[object_columns] = train_data[object_columns].astype('category')
train_data.dtypes

Age            int64
Gender      category
T_Bil        float64
D_Bil        float64
ALP          float64
ALT_GPT      float64
AST_GOT      float64
TP           float64
Alb          float64
AG_ratio     float64
disease      float64
dtype: object

In [155]:
# #ハイパーパラメータのチューニング
# import optuna
# from sklearn.metrics import mean_squared_error
# 
# # MeanF1スコアを格納するリスト
# mean_f1_scores = []
# 
# X = train_data.drop('health', axis=1).iloc[ : len(train)]
# y = train_data['health'].iloc[ : len(train)]
# 
# X_train, X_val, y_train, y_val = train_test_split(X,
#                                                   y, shuffle=True,train_size=0.7,
#                                                   random_state=seed, stratify=y)
# 
# 
# def objective(trial):
#     params = {
#         'objective': 'multiclass',
#         'metric': 'custom',
#         'random_seed': seed,
#         'num_class': 3,
#         'feature_fraction': trial.suggest_uniform('feature_fraction', 0.89704285838459497, 1.0),#0.9704285838459497, #
#         'num_leaves': trial.suggest_int('num_leaves', 2, 256),
#         'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
#         'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
#         #'max_bin': trial.suggest_int('max_bin', 255, 500),
#         'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
#         'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
#         'learning_rate': 0.05,
#         'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 5, 50),
#     
#     }
# 
#     train_lgb = lgb.Dataset(X_train, label=y_train)
#     val_lgb = lgb.Dataset(X_val, label=y_val)
# 
#     model = lgb.train(params, train_lgb, 
#                       valid_sets=[train_lgb, val_lgb],
#                       num_boost_round=100,
#                       callbacks=[lgb.early_stopping(stopping_rounds=10, 
#                                 verbose=False),lgb.log_evaluation(100)],
#                       # verbose_eval=10,
#                       feval=f1_macro,
#                       )
# 
#     y_pred_valid_prob = model.predict(X_val, num_iteration=model.best_iteration)
#     y_pred_valid = np.argmax(y_pred_valid_prob,axis = 1)
#     score = np.sqrt(mean_squared_error(y_val, y_pred_valid))
#     return score
# 
# study = optuna.create_study(sampler=optuna.samplers.RandomSampler(seed=seed))
# study.optimize(objective, n_trials=100)
# study.best_params

In [211]:
from sklearn.metrics import roc_auc_score
from sklearn import metrics

#交差検証

params = {
    'objective': 'binary',
    'metric': 'auc',
    'verbosity': -1,
    # 'n_estimators': 100,
    # 'boosting_type': 'gbdt',  # default = 'gbdt'
    # 'num_leaves': 244, #63,         # default = 31,
    'learning_rate': 0.1,    # default = 0.1
    # 'feature_fraction': 0.9356044384414135, #0.8,  # default = 1.0
    # 'bagging_freq': 5, #1,        # default = 0
    # 'bagging_fraction': 0.839196365086843, #0.8,  # default = 1.0
    'random_state': seed,
    # 'lambda_l1': 2.5361081166471375e-07,
    # 'lambda_l2': 2.5348407664333426e-07,
    # 'min_data_in_leaf': 7,

}

# クラスの比率が同じになるように分割
cv = StratifiedKFold(n_splits=5, random_state=seed, shuffle=True)

# AUCスコアを格納するリスト
AUC_scores = []

# モデルを保存するリスト
models = []

X = train_data.drop('disease', axis=1).iloc[: len(train)]
y = train_data['disease'].iloc[: len(train)]

for train_index, val_index in cv.split(X, y):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]
    
    # LightGBM用データセット
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_val = lgb.Dataset(X_val, y_val)

    # 一段目のlgb訓練
    lgb_model_first = lgb.train(params, lgb_train,
                                # num_boost_round=10000,
                                valid_sets=[lgb_train, lgb_val],
                                callbacks=[
                                    lgb.early_stopping(stopping_rounds=100, verbose=False),
                                    lgb.log_evaluation(100)],
                                )
    models.append(lgb_model_first)
    y_pred_prob_first = lgb_model_first.predict(X_train)
    y_val_pred_prob_first = lgb_model_first.predict(X_val)
    # forで2回目以降のLightGBMを繰り返してみる
    y_pred_prob = []
    y_val_pred_prob = []
    
    # 1段目の訓練データ予測とテストデータ予測を格納する
    y_pred_prob.append(pd.DataFrame(y_pred_prob_first))
    y_val_pred_prob.append(pd.DataFrame(y_val_pred_prob_first))
    
    repeat = 50  #50
    for _ in range(repeat):
        lgb_train = lgb.Dataset(y_pred_prob[-1], y_train)
        lgb_model = lgb.train(params, lgb_train, 
                              # num_boost_round=10000,
                              valid_sets=[lgb_train],
                              callbacks=[
                                  lgb.early_stopping(stopping_rounds=100, verbose=False),
                                  lgb.log_evaluation(100)],
                              # feval=f1_macro,
                              )
        #前のモデルが訓練データから予測した結果
        y_pred_prob.append(pd.DataFrame(lgb_model.predict(y_pred_prob[-1])))
    
        #前のモデルがテストデータから予測した結果
        y_val_pred_prob.append(pd.DataFrame(lgb_model.predict(y_val_pred_prob[-1])))
        
        models.append(lgb_model)
    
    # # 検証データで予測
    # y_pred_prob_first = pd.DataFrame(lgb_model_first.predict(X_val))  #, num_iteration=lgb.best_iteration)
    # y_pred_prob_second = lgb_model_second.predict(y_pred_prob_first)
    y_pred = np.where(y_val_pred_prob[-1] < 0.5, 0, 1)
# 
#     # y_pred = np.where(y_pred_prob_first < 0.5, 0, 1) # TODO　LGB1段にするにあたって追記
# 
    # AUC scoreを計算してリストに追加
    AUC_score = roc_auc_score(y_val, y_pred)
    AUC_scores.append(AUC_score)

# スコアを表示
print('Mean AUC scores:', AUC_scores)
print('Average AUC Score:', np.mean(AUC_scores))


[100]	training's auc: 1	valid_1's auc: 0.971639
[100]	training's auc: 0.999992
[100]	training's auc: 0.999992
[100]	training's auc: 0.999992
[100]	training's auc: 0.999992
[100]	training's auc: 0.999992
[100]	training's auc: 0.999992
[100]	training's auc: 0.999992
[100]	training's auc: 0.999992
[100]	training's auc: 0.999992
[100]	training's auc: 0.999992
[100]	training's auc: 0.999992
[100]	training's auc: 0.999992
[100]	training's auc: 0.999992
[100]	training's auc: 0.999992
[100]	training's auc: 0.999992
[100]	training's auc: 0.999992
[100]	training's auc: 0.999992
[100]	training's auc: 0.999992
[100]	training's auc: 0.999992
[100]	training's auc: 0.999992
[100]	training's auc: 0.999992
[100]	training's auc: 0.999992
[100]	training's auc: 0.999992
[100]	training's auc: 0.999992
[100]	training's auc: 0.999992
[100]	training's auc: 0.999992
[100]	training's auc: 0.999992
[100]	training's auc: 0.999992
[100]	training's auc: 0.999992
[100]	training's auc: 0.999992
[100]	training's auc: 

In [212]:
X_test = train_data.drop('disease', axis=1).iloc[len(train):]
lgb_train = lgb.Dataset(X, y)
# lgb_model_first = lgb.train(params, lgb_train, num_boost_round=100000,
#                             valid_sets=lgb_train,
#                             callbacks=[lgb.early_stopping(stopping_rounds=100,
#                                                           # verbose=True
#                                                           ), 
#                                        lgb.log_evaluation(100)],
#                             # feval=f1_macro,
#                             )
lgb_model_first = models[0]

y_pred_prob_first = pd.DataFrame(lgb_model_first.predict(X))
y_test_pred_prob_first = pd.DataFrame(lgb_model_first.predict(X_test))
# forで2回目以降のLightGBMを繰り返してみる
y_pred_prob = []
y_test_pred_prob = []

# 1段目の訓練データ予測とテストデータ予測を格納する
y_pred_prob.append(y_pred_prob_first)
y_test_pred_prob.append(y_test_pred_prob_first)

for i in range(repeat):
    lgb_train = lgb.Dataset(y_pred_prob[-1], y)
    # lgb_model = lgb.train(params, lgb_train, num_boost_round=10000,
    #                       valid_sets=[lgb_train],
    #                       callbacks=[
    #                           lgb.early_stopping(stopping_rounds=100, verbose=True),
    #                           lgb.log_evaluation(100)],
    #                       # feval=f1_macro,
    #                       )
    lgb_model = models[i+1]

    #前のモデルが訓練データから予測した結果
    y_pred_prob.append(pd.DataFrame(lgb_model.predict(y_pred_prob[-1])))

    #前のモデルがテストデータから予測した結果
    y_test_pred_prob.append(pd.DataFrame(lgb_model.predict(y_test_pred_prob[-1])))

y_pred = np.where(y_test_pred_prob[-1] < 0.5, 0, 1)

print('判定結果')
print('0:', np.count_nonzero(y_pred == 0), '1:', np.count_nonzero(y_pred == 1))

判定結果
0: 180 1: 202


In [213]:
submit = pd.DataFrame(train_data['disease'].iloc[len(train):])
# print(index)
# print(index.shape, y_pred.shape)
submit['disease'] = y_pred
print(submit)
# submit = pd.DataFrame(np.stack([index, y_pred], 1))
# y_pred
submit = submit.astype('int')
submit.to_csv('submit.csv', index=True, header=False)
submit

      disease
id           
891         0
892         1
893         0
894         1
895         0
896         1
897         0
898         0
899         0
900         0
901         1
902         0
903         1
904         1
905         0
906         0
907         1
908         1
909         1
910         0
911         0
912         0
913         0
914         0
915         1
916         0
917         0
918         1
919         1
920         0
921         1
922         0
923         0
924         1
925         0
926         0
927         1
928         1
929         0
930         0
931         1
932         0
933         0
934         0
935         0
936         1
937         1
938         1
939         0
940         0
941         1
942         1
943         1
944         0
945         1
946         0
947         0
948         0
949         1
950         0
951         0
952         0
953         0
954         1
955         1
956         0
957         0
958         1
959         1
960   

,disease
id,
891,0
892,1
893,0
894,1
895,0
896,1
897,0
898,0
899,0


In [143]:
# submission.csvの提出
# !signate submit --competition-id=1247 submission.csv --note LGB**51 # ここにコメント残せる

In [144]:
# # 一段目のLightGBMで訓練
# lgb_train = lgb.Dataset(X, y)
# lgb_model_first = lgb.train(params, lgb_train, num_boost_round=100000,
#                       valid_sets=lgb_train,
#                       callbacks=[lgb.early_stopping(stopping_rounds=100, 
#                             verbose=True),lgb.log_evaluation(100)],
#                       feval=f1_macro,
#                       )
# 
#     
# y_pred_prob_first = lgb_model_first.predict(X)
# 
# # 二段目のlgb訓練
# lgb_train_second = lgb.Dataset(y_pred_prob_first, y)
# lgb_model_second = lgb.train(params, lgb_train_second, num_boost_round=10000,
#                              valid_sets=[lgb_train_second],
#                              callbacks=[
#                                  lgb.early_stopping(stopping_rounds=100, verbose=True),
#                                  lgb.log_evaluation(100)],
#                              feval=f1_macro,
#                              )
# y_pred_prob_second = lgb_model_second.predict(y_pred_prob_first)
# 
# # 三段目のlgb訓練
# lgb_train_third = lgb.Dataset(y_pred_prob_second, y)
# lgb_model_third = lgb.train(params, lgb_train_third, num_boost_round=10000,
#                              valid_sets=[lgb_train_third],
#                              callbacks=[
#                                  lgb.early_stopping(stopping_rounds=100, verbose=True),
#                                  lgb.log_evaluation(100)],
#                              feval=f1_macro,
#                              )
# 
# 
# 
# 
# # LightGBMモデルが訓練データから予測したものを使って、
# 
# # testデータで予測
# X_test = train_data.drop('health', axis=1).iloc[len(train): ]
# y_test_pred_prob_first = lgb_model_first.predict(X_test) #, num_iteration=lgb
# # .best_iteration)
# y_test_pred_prob_second = lgb_model_second.predict(y_test_pred_prob_first)
# y_test_pred_prob_third = lgb_model_third.predict(y_test_pred_prob_second)
# y_pred = np.argmax(y_test_pred_prob_third,axis = 1)

In [145]:
# # lightGBMで訓練・予測の関数を作ってみる
# def lgb_spiral(X_train, y_train, X_test):
#     lgb_train = lgb.Dataset(X_train, y_train)
#     lgb_model = lgb.train(params, lgb_train, num_boost_round=10000,
#                              valid_sets=[lgb_train],
#                              callbacks=[
#                                  lgb.early_stopping(stopping_rounds=100, verbose=True),
#                                  lgb.log_evaluation(100)],
#                              feval=f1_macro,
#                              )
#     y_test = lgb_model.predict(X_test)
#     return y_test
# lgb_spiral(X, y, X_test)